In [1]:
from collections import Counter
from enum import IntEnum
from pathlib import Path
import pickle
from pprint import pprint
import shutil
import subprocess
from tqdm import tqdm
from typing import Dict, List, Optional

# Functionality Test about pywasm and wamr

Use examples in [python cookbook, 3rd Edition](https://www.amazon.com/Python-Cookbook-Third-David-Beazley/dp/1449340377) as test cases to verify the correctness of the implementation.

![](https://m.media-amazon.com/images/I/513ALClCHCL._SY445_SX342_.jpg)


In [2]:
IWASM = Path(shutil.which("iwasm"))

PY2WASM = Path(shutil.which("py2wasm"))

PYTHON_COOKBOOK = Path.cwd().joinpath("./cookbook/src")
assert PYTHON_COOKBOOK.exists(), "use `git submodule update` to fetch cookbook"


In [3]:
class CaseErrorCode(IntEnum):
    OK = 0
    COMPILATION_FAILURE = 1
    WASM_EXECUTION_FAILURE = 2
    PY_EXECUTION_FAILURE = 3
    DIFFERENT_RESULT = 4
    BYPASS = 5
    WASM_EXECUTION_TIMEOUT = 6
    PY_EXECUTION_TIMEOUT = 7

    def __str__(self):
        return f'{self.name}'

class CaseResult:
    def __init__(self, error_code: CaseErrorCode, msg: str):
        self.error_code = error_code
        self.msg = msg

    def __repr__(self):
        return f"{self.error_code}\n{self.msg}"


For each case, we are going to:

1. compile the python script to a WebAssembly core module
2. run the python script via python 3.11.9
3. run the WebAssembly core module via iwasm_wasi_interp_d3e89895b
4. compare both execution results of step 2 and step 3

*iwasm* needs to fully support wasm32-wasi. Its recommended compilation options are: `-DWAMR_BUILD_REF_TYPES=1 -DWAMR_BUILD_LIBC_BUILTIN=0`

In [4]:
def compile_py_2_wasm(py2wasm_bin: Path, py_file: Path, out_dir: Optional[Path]) -> Path:
    if not out_dir is None:
        out_dir.mkdir(parents=True, exist_ok=True)
        wasm_file = out_dir.joinpath(py_file.stem + ".wasm")
    else:
        wasm_file = py_file.with_suffix(".wasm")
    
    cmd = f"{py2wasm_bin} {py_file} -o {wasm_file}"
    subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True, timeout=120)
    return wasm_file

def execute_py(py_file: Path) -> subprocess.CompletedProcess:
    """
    Since relative pathes are used in examples.
    """
    cwd = py_file.parent
    cmd = f"python {py_file}"
    return subprocess.run(cmd, cwd=cwd, capture_output=True, check=True, shell=True, universal_newlines=True, text=True, timeout=30)

def execute_wasm(iwasm_bin: Path, wasm_file: Path) -> subprocess.CompletedProcess:
    """
    To avoid the WASI pre-opens problem, run under the same directory with the wasm file
    """
    cwd = wasm_file.parent
    # stack: 128M  pre-open: PWD
    cmd = f"{iwasm_bin} --dir=. --stack-size=134217728 {wasm_file.relative_to(cwd)}"
    return subprocess.run(cmd, cwd=cwd, capture_output=True, check=True, shell=True, universal_newlines=True, text=True, timeout=80)
    

def compare_result(expected: str, actual: str) -> bool:
    return expected == actual

##################################################
def execute_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    ## execute python firstly to avoid un-necessary wasm compilation
    try:
        p = execute_py(py_file)
    except subprocess.CalledProcessError as e:
        return CaseResult(CaseErrorCode.PY_EXECUTION_FAILURE, e.stderr if e.stderr else e.stdout)
    except subprocess.TimeoutExpired as e:
        return CaseResult(CaseErrorCode.PY_EXECUTION_TIMEOUT, e.stderr if e.stderr else e.stdout)
    output_from_py = p.stdout

    try:
        wasm_file = compile_py_2_wasm(PY2WASM, py_file, out_dir)
    except subprocess.CalledProcessError as e:
        return CaseResult(CaseErrorCode.COMPILATION_FAILURE, e.stderr if e.stderr else e.stdout)
    except subprocess.TimeoutExpired as e:
        return CaseResult(CaseErrorCode.COMPILATION_FAILURE, e.stderr if e.stderr else e.stdout)

    try:
        p = execute_wasm(IWASM, wasm_file)
    except subprocess.CalledProcessError as e:
        return CaseResult(CaseErrorCode.WASM_EXECUTION_FAILURE, f"{e.stderr}\n{e.stdout}")
    except subprocess.TimeoutExpired as e:
        return CaseResult(CaseErrorCode.WASM_EXECUTION_TIMEOUT, f"{e.stderr}\n{e.stdout}")
    output_from_wasm = p.stdout

    if not compare_result(output_from_py, output_from_wasm):
        return CaseResult(CaseErrorCode.DIFFERENT_RESULT, f"{'>' * 40}\n{output_from_py}{'-' * 20} V.S. {'-' * 20}\n{output_from_wasm}{'<' * 40}")
    
    return CaseResult(CaseErrorCode.OK, "")

def visit_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    print(f"py_file={py_file}, out_dir={out_dir}")
    return CaseResult(CaseErrorCode.OK, "")

def execute_test(chapter: Path, bypass: List[str]) -> Dict[str, CaseResult]:
    # list all sub-directories in chapter
    chapter_tests = [x for x in chapter.iterdir() if x.is_dir()]

    # walk
    ret = {}
    for test_dir in tqdm(chapter_tests):
        # assume all .py under the test_dir are test cases
        py_files = [x for x in test_dir.iterdir() if x.suffix == ".py"]

        for py_file in py_files:
            case_name = f"{test_dir.name}.{py_file.stem}"

            if case_name in bypass:
                ret[case_name] = CaseResult(CaseErrorCode.BYPASS, "")
                continue

            result = execute_case(py_file, None)
            #result = visit_case(py_file, None)
            ret[case_name] = result 
    
    return ret

def summarize_result(results: Dict[str, CaseResult]) -> None:
    counter = Counter([x.error_code for x in results.values()])
    print()
    print(counter)
    print()

    # print failures
    for k,v in results.items():
        if v.error_code != CaseErrorCode.OK:
            print(f"{k} FAILED:\n{v}")
            print()

def stash_result(results: Dict[str, CaseResult], out_file: Path) -> None:
    with out_file.open("wb") as f:
        pickle.dump(results, f)

def restore_result(out_file: Path) -> Dict[str, CaseResult]:
    with out_file.open("rb") as f:
        return pickle.load(f)

## Bypass

Bypass some cases that are not supported by the current implementation.
- `socket`. because there is no official support for socket in wasi-libc. *wasip1*
- `subproess`. 

In [5]:
# find . -name "*.py" -type f | xargs grep -rin "socket" | awk -F':' '{print $1}' | sort | uniq
SOCKET_INVOLVED_CASES = {
    "5": [
        "wrapping_an_existing_file_descriptor_as_a_file_object.echo",
    ],
    "7": [
        "making_an_n-argument_callable_work_as_a_callable_with_fewer_arguments.example3",
    ],
    "8": [
        "how_to_define_an_interface_or_abstract_base_class.example",
        "making_objects_support_the_context_manager_protocol.example1",
        "making_objects_support_the_context_manager_protocol.example2",
    ],
    "11": [
        "adding_ssl_to_network_servers.echoclient",
        "adding_ssl_to_network_servers.echoserv",
        "adding_ssl_to_network_servers.sslmixin",
        "creating_a_tcp_server.echoclient",
        "creating_a_tcp_server.echoserv",
        "creating_a_tcp_server.echoserv1",
        "creating_a_tcp_server.echoserv2",
        "creating_a_tcp_server.echoserv3",
        "creating_a_tcp_server.echoserv4",
        "creating_a_tcp_server.echoserv5",
        "creating_a_tcp_server.threadedserv",
        "creating_a_udp_server.client",
        "creating_a_udp_server.timeserv1",
        "creating_a_udp_server.timeserv2",
        "event_driven_io_explained.tcpclient",
        "event_driven_io_explained.tcpserver",
        "event_driven_io_explained.threadpool",
        "event_driven_io_explained.thrpoolclient",
        "event_driven_io_explained.udpclient",
        "event_driven_io_explained.udpserver",
        "passing_a_socket_file_descriptor_between_processes.client1",
        "passing_a_socket_file_descriptor_between_processes.server",
        "passing_a_socket_file_descriptor_between_processes.server1",
        "passing_a_socket_file_descriptor_between_processes.servermp",
        "passing_a_socket_file_descriptor_between_processes.worker",
        "passing_a_socket_file_descriptor_between_processes.workermp",
        "simple_authentication_of_clients.client",
        "simple_authentication_of_clients.server",
        "zero_copy_sending_and_receiving_of_large_arrays.client",
        "zero_copy_sending_and_receiving_of_large_arrays.server",
    ],
    "12": [
        "how_to_create_a_thread_pool.example1",
        "how_to_create_a_thread_pool.example2",
        "polling_multiple_thread_queues.pqueue",
        "storing_thread_specific_state.example1",
        "storing_thread_specific_state.example2",
        "using_generators_as_an_alternative_to_threads.netsched",
    ],
    "13": [
        "generating_a_range_of_ip_addresses_from_a_cidr_address.example"
    ]
}

# find . -name "*.py" -type f | xargs grep -rin "subprocess" | awk -F':' '{print $1}' | sort | uniq
SUBPROCESS_INVOLVED_CASES = {
    "13": [
        "executing_an_external_command_and_getting_its_output.example1",
        "executing_an_external_command_and_getting_its_output.example2"
    ]
}

# find . -name "*.py" -type f | xargs grep -rin "multiprocess" | awk -F':' '{print $1}' | sort | uniq
MULTIPROCESS_INVOLVED_CASES = {
    "7": [
        "inlining_callback_functions.example",
        "making_an_n-argument_callable_work_as_a_callable_with_fewer_arguments.example2"
    ],
    "11": [
        "implementing_remote_procedure_call.jsonrpclient",
        "implementing_remote_procedure_call.jsonrpcserver",
        "implementing_remote_procedure_call.rpcclient",
        "implementing_remote_procedure_call.rpcserver",
        "passing_a_socket_file_descriptor_between_processes.server1",
        "passing_a_socket_file_descriptor_between_processes.servermp",
        "passing_a_socket_file_descriptor_between_processes.workermp",
        "simple_communication_between_interpreters.echoclient",
        "simple_communication_between_interpreters.echoserv",
    ]
}

def get_ignore_list(N: str) -> List[str]:
    """
    ZZZ_CASE.extend(XXX_CASE).extend(YYY_CASE)
    """
    return SOCKET_INVOLVED_CASES.get(N, []) \
        + SUBPROCESS_INVOLVED_CASES.get(N, []) \
        + MULTIPROCESS_INVOLVED_CASES.get(N, [])

## Chapter 1. Data Structures And Algorithms

`DIFFERENT_RESULT` of _finding_out_what_two_dictionaries_have_in_common_ isn't a bug 😄. It is caused by the different order of the keys in the dictionary.

`EXECUTION_FAILURE` of _transforming_and_reducing_data_at_the_same_time_ is a requestd feature that isn't supported by the current implementation of WAMR

---

**AR ✨**

- [ ] WAMR needs to support `~`, `.`, `..`


In [6]:
chapter_1 = PYTHON_COOKBOOK.joinpath("1")
chapter_1_result = execute_test(chapter_1, get_ignore_list("1"))
summarize_result(chapter_1_result)
stash_result(chapter_1_result, Path("chapter_1_result.pkl"))

100%|██████████| 16/16 [05:12<00:00, 19.56s/it]


Counter({<CaseErrorCode.OK: 0>: 15, <CaseErrorCode.DIFFERENT_RESULT: 4>: 1, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 1})

finding_out_what_two_dictionaries_have_in_common.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Common keys: {'y', 'x'}
Keys in a not in b: {'z'}
(key,value) pairs in common: {('y', 2)}
-------------------- V.S. --------------------
Common keys: {'x', 'y'}
Keys in a not in b: {'z'}
(key,value) pairs in common: {('y', 2)}
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

transforming_and_reducing_data_at_the_same_time.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 6, in <module>
    files = os.listdir(os.path.expanduser('~'))
    
FileNotFoundError: [Errno 44] No such file or directory: '~'





## Chapter 2. Strings And Text

There are `try...except...` in some cases. Need to deep dive into the wasm implementation.


In [7]:
chapter_2 = PYTHON_COOKBOOK.joinpath("2")
chapter_2_result = execute_test(chapter_2, get_ignore_list("2"))
summarize_result(chapter_2_result)
stash_result(chapter_2_result, Path("chapter_2_result.pkl"))

100%|██████████| 13/13 [04:19<00:00, 19.97s/it]


Counter({<CaseErrorCode.OK: 0>: 15})



## Chapter 3. Numbers, Dates, And Times


In [8]:
chapter_3 = PYTHON_COOKBOOK.joinpath("3")
chapter_3_result = execute_test(chapter_3, get_ignore_list("3"))
summarize_result(chapter_3_result)
stash_result(chapter_3_result, Path("chapter_3_result.pkl"))

100%|██████████| 2/2 [00:34<00:00, 17.07s/it]


Counter({<CaseErrorCode.OK: 0>: 2})



## Chapter 4. Iterators And Generators


In [9]:
chapter_4 = PYTHON_COOKBOOK.joinpath("4")
chapter_4_result = execute_test(chapter_4, get_ignore_list("4"))
summarize_result(chapter_4_result)
stash_result(chapter_4_result, Path("chapter_4_result.pkl"))

100%|██████████| 10/10 [02:26<00:00, 14.64s/it]


Counter({<CaseErrorCode.OK: 0>: 9, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 1, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 1})

creating_data_processing_pipelines.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 3, in <module>
    import gzip
    
  File "./gzip.py", line 9, in <module>
ImportError: dynamic libraries are not implemented in wasi



how_to_flatten_a_nested_sequence.example FAILED:
PY_EXECUTION_FAILURE
Traceback (most recent call last):
  File "/workspaces/wamr-with-py/cookbook/src/4/how_to_flatten_a_nested_sequence/example.py", line 3, in <module>
    from collections import Iterable
ImportError: cannot import name 'Iterable' from 'collections' (/usr/local/lib/python3.11/collections/__init__.py)




## Chapter 5. Files And I/O


In [10]:
chapter_5 = PYTHON_COOKBOOK.joinpath("5")
chapter_5_result = execute_test(chapter_5, get_ignore_list("5"))
summarize_result(chapter_5_result)
stash_result(chapter_5_result, Path("chapter_5_result.pkl"))

100%|██████████| 6/6 [01:03<00:00, 10.65s/it]


Counter({<CaseErrorCode.OK: 0>: 4, <CaseErrorCode.BYPASS: 5>: 1, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 1})

wrapping_an_existing_file_descriptor_as_a_file_object.echo FAILED:
BYPASS


adding_or_changing_the_encoding_of_an_already_open_file.example FAILED:
PY_EXECUTION_FAILURE
Traceback (most recent call last):
  File "/workspaces/wamr-with-py/cookbook/src/5/adding_or_changing_the_encoding_of_an_already_open_file/example.py", line 8, in <module>
    text = f.read()
           ^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte




## Chapter 6. Data Encoding And Processing


In [11]:
chapter_6 = PYTHON_COOKBOOK.joinpath("6")
chapter_6_result = execute_test(chapter_6, get_ignore_list("6"))
summarize_result(chapter_6_result)
stash_result(chapter_6_result, Path("chapter_6_result.pkl"))

100%|██████████| 8/8 [03:52<00:00, 29.09s/it]


Counter({<CaseErrorCode.OK: 0>: 10, <CaseErrorCode.DIFFERENT_RESULT: 4>: 2, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 1, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 1})

reading_and_writing_json_data.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
ACME
50
490.1
{"__classname__": "Point", "x": 3, "y": 4}
3
4
-------------------- V.S. --------------------
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
ACME
50
490.1
{"__classname__": "Point", "x": 3, "y": 4}
3
4
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

parsing_xml_documents_with_namespaces.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<Element '{http://www.w3.org/1999/xhtml}html' at 0x7fa2c83fcfe0>
Hello World
-------------------- V.S. --------------------
<Element '{http://www.w3.org/1999/xhtml}html' at 0xc776b8>
Hello World
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

parsing_simple_xml_data.example FAI

## Chapter 7. Functions


In [12]:
chapter_7 = PYTHON_COOKBOOK.joinpath("7")
chapter_7_result = execute_test(chapter_7, get_ignore_list("7"))
summarize_result(chapter_7_result)
stash_result(chapter_7_result, Path("chapter_7_result.pkl"))

100%|██████████| 7/7 [02:13<00:00, 19.09s/it]


Counter({<CaseErrorCode.OK: 0>: 6, <CaseErrorCode.BYPASS: 5>: 3, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 2})

inlining_callback_functions.example FAILED:
BYPASS


making_an_n-argument_callable_work_as_a_callable_with_fewer_arguments.example2 FAILED:
BYPASS


making_an_n-argument_callable_work_as_a_callable_with_fewer_arguments.example3 FAILED:
BYPASS


accessing_variables_defined_inside_a_closure.example2 FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example2.py", line 34, in <module>
    s.push(10)

AttributeError: 'ClosureInstance' object has no attribute 'push'



accessing_variables_defined_inside_a_closure.example3 FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example3.py", line 27, in <module>
    print(timeit('s.push(1); s.pop()', 'from __main__ import s'))

  File "./timeit.py", line 237, in timeit
  File "./timeit.py", line 180, in timeit
  File "<timeit-src>", line 6, in inner
AttributeError: 'ClosureInstance' ob

## Chapter 8. Classes And Objects


In [13]:
chapter_8 = PYTHON_COOKBOOK.joinpath("8")
chapter_8_result = execute_test(chapter_8, get_ignore_list("8"))
summarize_result(chapter_8_result)
stash_result(chapter_8_result, Path("chapter_8_result.pkl"))

100%|██████████| 24/24 [12:31<00:00, 31.30s/it]


Counter({<CaseErrorCode.OK: 0>: 40, <CaseErrorCode.BYPASS: 5>: 3, <CaseErrorCode.DIFFERENT_RESULT: 4>: 3, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 2, <CaseErrorCode.WASM_EXECUTION_TIMEOUT: 6>: 1})

making_objects_support_the_context_manager_protocol.example2 FAILED:
BYPASS


making_objects_support_the_context_manager_protocol.example1 FAILED:
BYPASS


implementing_the_visitor_pattern_without_recursion.example1 FAILED:
WASM_EXECUTION_TIMEOUT
None
None

how_to_define_an_interface_or_abstract_base_class.example FAILED:
BYPASS


creating_an_instance_without_invoking_init.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
False
2012
8
2024 7 27
-------------------- V.S. --------------------
False
2012
8
2024 7 27
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

implementing_stateful_objects_or_state_machines.example2 FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Not open
reading
-------------------- V.S. --------------------
Not open
reading
<<<<<<<<<<<<

## Chapter 9. Metaprogramming


In [14]:
chapter_9 = PYTHON_COOKBOOK.joinpath("9")
chapter_9_result = execute_test(chapter_9, get_ignore_list("9"))
summarize_result(chapter_9_result)
stash_result(chapter_9_result, Path("chapter_9_result.pkl"))

100%|██████████| 23/23 [09:26<00:00, 24.64s/it]


Counter({<CaseErrorCode.OK: 0>: 22, <CaseErrorCode.DIFFERENT_RESULT: 4>: 10, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 2, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 2})

preserving_function_metadata_when_writing_decorators.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
countdown 0.0022516250610351562
Name: countdown
Docstring: '\n        Counts down\n        '
Annotations: {'n': <class 'int'>}
-------------------- V.S. --------------------
countdown 0.008459806442260742
Name: countdown
Docstring: '\n        Counts down\n        '
Annotations: {'n': <class 'int'>}
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

defining_decorators_as_classes.example2 FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
5
9
ncalls: 2
<__main__.Spam object at 0x7f5893afee10> 1
<__main__.Spam object at 0x7f5893afee10> 2
<__main__.Spam object at 0x7f5893afee10> 3
ncalls: 3
-------------------- V.S. --------------------
5
9
ncalls: 2
<__main__.Spam object at 0xbf1a20> 1
<__m

## Chapter 10. Modules And Packages


In [15]:
chapter_10 = PYTHON_COOKBOOK.joinpath("10")
chapter_10_result = execute_test(chapter_10, get_ignore_list("10"))
summarize_result(chapter_10_result)
stash_result(chapter_10_result, Path("chapter_10_result.pkl"))

100%|██████████| 4/4 [01:40<00:00, 25.11s/it]


Counter({<CaseErrorCode.OK: 0>: 5, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 3, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 1})

loading_modules_from_a_remote_machine_using_import_hooks.metaexample FAILED:
PY_EXECUTION_FAILURE
Traceback (most recent call last):
  File "/workspaces/wamr-with-py/cookbook/src/10/loading_modules_from_a_remote_machine_using_import_hooks/metaexample.py", line 13, in <module>
    import fib
ModuleNotFoundError: No module named 'fib'


loading_modules_from_a_remote_machine_using_import_hooks.pathexample FAILED:
PY_EXECUTION_FAILURE
Traceback (most recent call last):
  File "/workspaces/wamr-with-py/cookbook/src/10/loading_modules_from_a_remote_machine_using_import_hooks/pathexample.py", line 14, in <module>
    import fib
ModuleNotFoundError: No module named 'fib'


loading_modules_from_a_remote_machine_using_import_hooks.urlimport FAILED:
WASM_EXECUTION_FAILURE
./urlimport.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib and slated

## Chapter 11. Network And Web Programming

In [16]:
chapter_11 = PYTHON_COOKBOOK.joinpath("11")
chapter_11_result = execute_test(chapter_11, get_ignore_list("11"))
summarize_result(chapter_11_result)
stash_result(chapter_11_result, Path("chapter_11_result.pkl"))

100%|██████████| 12/12 [04:48<00:00, 24.01s/it]


Counter({<CaseErrorCode.BYPASS: 5>: 36, <CaseErrorCode.OK: 0>: 4, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 4, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 3, <CaseErrorCode.PY_EXECUTION_TIMEOUT: 7>: 3})

event_driven_io_explained.tcpclient FAILED:
BYPASS


event_driven_io_explained.threadpool FAILED:
BYPASS


event_driven_io_explained.udpclient FAILED:
BYPASS


event_driven_io_explained.thrpoolclient FAILED:
BYPASS


event_driven_io_explained.tcpserver FAILED:
BYPASS


event_driven_io_explained.udpserver FAILED:
BYPASS


interacting_with_http_services_as_a_client.example4 FAILED:
PY_EXECUTION_FAILURE
Traceback (most recent call last):
  File "/workspaces/wamr-with-py/cookbook/src/11/interacting_with_http_services_as_a_client/example4.py", line 8, in <module>
    last_modified =	resp.headers['last-modified']
                    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/requests/structures.py", line 52, in __getitem__
    return self._store[key

## Chapter 12. Concurrency


In [17]:
chapter_12 = PYTHON_COOKBOOK.joinpath("12")
chapter_12_result = execute_test(chapter_12, get_ignore_list("12"))
summarize_result(chapter_12_result)
stash_result(chapter_12_result, Path("chapter_12_result.pkl"))

100%|██████████| 13/13 [07:59<00:00, 36.86s/it]


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 10, <CaseErrorCode.BYPASS: 5>: 6, <CaseErrorCode.OK: 0>: 5, <CaseErrorCode.PY_EXECUTION_TIMEOUT: 7>: 5, <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 2})

how_to_start_and_stop_threads.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 19, in <module>
    t.start()
    
  File "./threading.py", line 964, in start
RuntimeError: can't start new thread



defining_an_actor_task.actor FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./actor.py", line 70, in <module>
    p.start()

  File "./actor.py", line 40, in start
    t.start()

  File "./threading.py", line 964, in start
RuntimeError: can't start new thread



defining_an_actor_task.tagged FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./tagged.py", line 19, in <module>
    a.start()

  File "./actor.py", line 40, in start
    t.start()

  File "./threading.py", line 964, in start
Runt

## Chatper 13. Utility Scripting And System Administration


In [18]:
chapter_13 = PYTHON_COOKBOOK.joinpath("13")
chapter_13_result = execute_test(chapter_13, get_ignore_list("13"))
summarize_result(chapter_13_result)
stash_result(chapter_13_result, Path("chapter_13_result.pkl"))

100%|██████████| 11/11 [02:08<00:00, 11.66s/it]


Counter({<CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 4, <CaseErrorCode.OK: 0>: 4, <CaseErrorCode.BYPASS: 5>: 3, <CaseErrorCode.PY_EXECUTION_TIMEOUT: 7>: 1, <CaseErrorCode.DIFFERENT_RESULT: 4>: 1})

parsing_command_line_options.search FAILED:
PY_EXECUTION_FAILURE
usage: search.py [-h] -p pattern [-v] [-o OUTFILE] [--speed {fast,slow}]
                 [filename ...]
search.py: error: the following arguments are required: -p/--pat


prompting_for_a_password_at_runtime.example FAILED:
PY_EXECUTION_TIMEOUT
b'/usr/local/lib/python3.11/getpass.py:91: GetPassWarning: Can not control echo on the terminal.\n  passwd = fallback_getpass(prompt, stream)\nWarning: Password input may be echoed.\nPassword: '

generating_a_range_of_ip_addresses_from_a_cidr_address.example FAILED:
BYPASS


putting_limits_on_memory_and_cpu_usage.example FAILED:
PY_EXECUTION_FAILURE
Time's up!


finding_files.modified_within FAILED:
PY_EXECUTION_FAILURE
Usage: /workspaces/wamr-with-py/cookbook/src/13/finding_files/modified

## Chapter 14. Testing, Debugging, And Exceptions


In [19]:
chapter_14 = PYTHON_COOKBOOK.joinpath("14")
chapter_14_result = execute_test(chapter_14, get_ignore_list("14"))
summarize_result(chapter_14_result)
stash_result(chapter_14_result, Path("chapter_14_result.pkl"))

100%|██████████| 7/7 [04:04<00:00, 34.98s/it]


Counter({<CaseErrorCode.OK: 0>: 4, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 2, <CaseErrorCode.WASM_EXECUTION_TIMEOUT: 6>: 1, <CaseErrorCode.DIFFERENT_RESULT: 4>: 1})

logging_test_output_to_a_file.test FAILED:
WASM_EXECUTION_FAILURE

Exception: indirect call type mismatch


make_your_programs_run_faster.example FAILED:
WASM_EXECUTION_TIMEOUT
None
None

testing_for_exceptional_conditions_in_unit_tests.test FAILED:
WASM_EXECUTION_FAILURE

Exception: indirect call type mismatch


profiling_and_timing_your_program.timethis FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
__main__.countdown : 0.22145235899370164
-------------------- V.S. --------------------
__main__.countdown : 0.8869439359987155
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<



## Chapter 15. C Extensions


In [20]:
chapter_15 = PYTHON_COOKBOOK.joinpath("15")
chapter_15_result = execute_test(chapter_15, get_ignore_list("15"))
summarize_result(chapter_15_result)
stash_result(chapter_15_result, Path("chapter_15_result.pkl"))

100%|██████████| 16/16 [00:18<00:00,  1.13s/it]


Counter({<CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 30, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 1})

working_with_c_strings_of_dubious_encoding.setup FAILED:
PY_EXECUTION_FAILURE
usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.py --help [cmd1 cmd2 ...]
   or: setup.py --help-commands
   or: setup.py cmd --help

error: no commands supplied


working_with_c_strings_of_dubious_encoding.example FAILED:
PY_EXECUTION_FAILURE
Traceback (most recent call last):
  File "/workspaces/wamr-with-py/cookbook/src/15/working_with_c_strings_of_dubious_encoding/example.py", line 1, in <module>
    import sample
ModuleNotFoundError: No module named 'sample'


passing_null_terminated_strings_to_c_libraries.setup FAILED:
PY_EXECUTION_FAILURE
usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.py --help [cmd1 cmd2 ...]
   or: setup.py --help-commands
   or: setup.py cmd --help

error: no commands supplied


passing_null_terminated_strings_t

## Summary


In [21]:
RERUN = True

if RERUN :
    chapter_x_results = {
        **chapter_1_result,
        **chapter_2_result,
        **chapter_3_result,
        **chapter_4_result,
        **chapter_5_result,
        **chapter_6_result,
        **chapter_7_result,
        **chapter_8_result,
        **chapter_9_result,
        **chapter_10_result,
        **chapter_11_result,
        **chapter_12_result,
        **chapter_13_result,
        **chapter_14_result,
        **chapter_15_result,
    }
else:
    # instead of a re-run, we can load the previous results
    chapter_x_results = restore_result(Path("cookbook_result.pkl"))

x_counters = Counter([x.error_code for x in chapter_x_results.values()])
print()
pprint(x_counters)
print()

stash_result(chapter_x_results, Path("cookbook_result.pkl"))



Counter({<CaseErrorCode.OK: 0>: 145,
         <CaseErrorCode.BYPASS: 5>: 52,
         <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 49,
         <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 25,
         <CaseErrorCode.DIFFERENT_RESULT: 4>: 18,
         <CaseErrorCode.PY_EXECUTION_TIMEOUT: 7>: 9,
         <CaseErrorCode.WASM_EXECUTION_TIMEOUT: 6>: 2})



In [22]:
ALL_CASES_NUM = len(chapter_x_results.keys())
VALID_CASES_NUM = ALL_CASES_NUM  \
    - x_counters.get(CaseErrorCode.PY_EXECUTION_FAILURE) \
    - x_counters.get(CaseErrorCode.PY_EXECUTION_TIMEOUT) \
    - x_counters.get(CaseErrorCode.BYPASS)

FAILED_RATE = (x_counters.get(CaseErrorCode.WASM_EXECUTION_FAILURE) +
               x_counters.get(CaseErrorCode.WASM_EXECUTION_TIMEOUT)) / VALID_CASES_NUM / 1.0
print(f"FAILED: {FAILED_RATE:.2%}")

FAILED: 14.21%


Since the requirement, we don't need to support all the features of python. We willn't count such chapters:
- Chapter 11. Network And Web Programming
- Chapter 12. Concurrency
- Chapter 13. Utility Scripting And System Administration
- Chapter 15. C Extensions

In [23]:
chapter_xxx_results = {
    **chapter_1_result,
    **chapter_2_result,
    **chapter_3_result,
    **chapter_4_result,
    **chapter_5_result,
    **chapter_6_result,
    **chapter_7_result,
    **chapter_8_result,
    **chapter_9_result,
    **chapter_10_result,
    **chapter_14_result,
}


x_counters = Counter([x.error_code for x in chapter_xxx_results.values()])
print()
pprint(x_counters)
print()



Counter({<CaseErrorCode.OK: 0>: 132,
         <CaseErrorCode.DIFFERENT_RESULT: 4>: 17,
         <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 10,
         <CaseErrorCode.PY_EXECUTION_FAILURE: 3>: 10,
         <CaseErrorCode.BYPASS: 5>: 7,
         <CaseErrorCode.WASM_EXECUTION_TIMEOUT: 6>: 2})



In [24]:
ALL_CASES_NUM = len(chapter_xxx_results.keys())
VALID_CASES_NUM = ALL_CASES_NUM  \
    - x_counters.get(CaseErrorCode.PY_EXECUTION_FAILURE, 0) \
    - x_counters.get(CaseErrorCode.PY_EXECUTION_TIMEOUT, 0) \
    - x_counters.get(CaseErrorCode.BYPASS, 0)

FAILED_RATE = (x_counters.get(CaseErrorCode.WASM_EXECUTION_FAILURE) +
               x_counters.get(CaseErrorCode.WASM_EXECUTION_TIMEOUT)) / VALID_CASES_NUM / 1.0
print(f"FAILED: {FAILED_RATE:.2%}")

FAILED: 7.45%
